In [17]:
pip install scikit-surprise

In [18]:
import pandas as pd
from surprise import KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

In [19]:
rating_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/ratings.csv"
rating_df = pd.read_csv(rating_url)

In [20]:
rating_df.head()

,user,item,rating
0,1889878,CC0101EN,3.0
1,1342067,CL0101EN,3.0
2,1990814,ML0120ENv3,3.0
3,380098,BD0211EN,3.0
4,779563,DS0101EN,3.0


In [21]:
rating_df.shape

(233306, 3)

In [22]:
rating_df = rating_df[:50000]

In [23]:
rating_sparse_df = rating_df.pivot(index = 'user', columns = 'item', values = 'rating').fillna(0).reset_index().rename_axis(index = None, columns = None)
rating_sparse_df.head()

,user,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,...,SW0101EN,SW0201EN,TA0105,TA0105EN,TA0106EN,TMP0105EN,TMP0106,TMP107,WA0101EN,WA0103EN
0,2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,4,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
#Example
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k', prompt=False)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous KNNBasic algorithm.
algo = KNNBasic()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9817


0.9816626097616735

In [25]:
rating_df.to_csv("course_ratings.csv", index=False)
reader = Reader(line_format='user item rating', sep=',', skip_lines=1, rating_scale=(2, 3))
coruse_dataset = Dataset.load_from_file("course_ratings.csv", reader=reader)
coruse_dataset

In [26]:
trainset, testset = train_test_split(coruse_dataset, test_size=.3)

In [27]:
print(f"Total {trainset.n_users} users and {trainset.n_items} items in the trainingset")

Total 19190 users and 118 items in the trainingset


In [28]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}
bsl_options = {'method':'als', 'learning_rate':'0.005', 'n_epochs': 20,}
knn = KNNBasic(max_k = 10, min_k = 5, sim_options = sim_options, bsl_options = bsl_options)
knn.fit(trainset)
pred = knn.test(testset)
accuracy.rmse(pred)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.2164


0.2163610286587419